## Part 1: Setup

### Install Packages

In [1]:
# Import Packages
import praw, time, os, pyarrow
from IPython.display import display
from dotenv import load_dotenv, dotenv_values
from requests import Session
import pandas as pd

# Load environment variables from .env file
load_dotenv('.env')
config = dotenv_values()

## Part 2: Collecting Data from Reddit

### Open Reddit Connection

In [2]:
# Create a custom session with a timeout
session = Session()
session.headers.update({'User-Agent': 'praw'})
session.timeout = 10  # Set a timeout of 10 seconds

# Login to Reddit using PRAW
reddit = praw.Reddit(
    client_id=config['CLIENT_ID'],
    client_secret=config['CLIENT_SECRET'],
    requestor_kwargs={"session": session},
    username=config['USERNAME'],
    password=config['PASSWORD'],
    user_agent="CS470 ML Project Access by u/GregorybLafayetteML"
)

# Add some peripheral config data
reddit.config.log_requests = 1
reddit.config.store_json_result = True

# Test the connection
try:
    username = reddit.user.me()
    print("Successfully logged in to Reddit!")
    print(f"Logged in as: u/{username}")
except Exception as e:
    print(f"Failed to log in: {e}")

Successfully logged in to Reddit!
Logged in as: u/GregorybLafayetteML


### Accessing Reddit Data

To access reddit posts, we'll need send a request with the number of post we want to get. The following example finds the top 10 hottest posts on the u/wallstreetbets subreddit. We'll show the post title, score, flair, and URL. 

In [3]:
top_posts = reddit.subreddit('wallstreetbets').hot(limit=10)
print("Top 10 hot posts from r/wallstreetbets:")
for post in top_posts:
    print(f"Title: {post.title}, Score: {post.score}, Flair: {post.link_flair_text}, URL: {post.url}")

Top 10 hot posts from r/wallstreetbets:
Title: What Are Your Moves Tomorrow, April 10, 2025, Score: 310, Flair: Daily Discussion, URL: https://www.reddit.com/r/wallstreetbets/comments/1jvf5zp/what_are_your_moves_tomorrow_april_10_2025/
Title: Weekly Earnings Thread 4/7 - 4/11, Score: 144, Flair: Earnings Thread, URL: https://i.redd.it/3sw567xwstse1.jpeg
Title: Tinfoil hat alert: Looks like insiders got a 20m head start for today’s face ripper rally, Score: 35930, Flair: News, URL: https://i.redd.it/pg8pcxd90vte1.jpeg
Title: Hold onto your butts, Score: 28097, Flair: Gain, URL: https://i.redd.it/ewzhovurnute1.jpeg
Title: BREAKING NEWS: Trump Says Tariffs Paused for 90 Days on Non-Retaliating Countries, Score: 28556, Flair: News, URL: https://www.bloomberg.com/news/live-blog/2025-04-08/trump-tariffs-stock-market-updates?srnd=homepage-europe&embedded-checkout=true
Title: World Record %???, Score: 3672, Flair: Gain, URL: https://www.reddit.com/gallery/1jvhrqj
Title: Lost life savings, dad 

For this project, we'll need far more than ten posts at a time. The reddit API will limit our access to 100 posts at a time. Fortunately, the api uses a ListingGenerator which allows us to access our metered connection in sequential blocks. The following example shows how we can utilize this behavior, grabbing blocks of 100 posts at a time. In our example, we'll grab blocks of posts until we reach 5000 posts or our access times out. Notice that the procedure ends early with around 750-800 posts collected. The results are sparce, because our connection either timed out or was metered down by reddit. The latter option is more likely.

In [4]:
# Access the subreddit
subreddit = reddit.subreddit("wallstreetbets")

# Initialize variables
batch_size = 50 # Number of posts per batch
total_posts = 5000  # Total number of posts to fetch
all_posts = []  # To store all the retrieved posts
after = None  # To keep track of the last post for pagination

# Fetch posts in batches
while len(all_posts) < total_posts:
    # Fetch the next batch of posts
    submissions = subreddit.new(limit=batch_size, params={"after": after})
    
    batch_posts = []
    for submission in submissions:
        batch_posts.append(submission)

        # Update the `after` variable with the last submission's fullname
        after = submission.fullname

    # Add the batch to the main list
    all_posts.extend(batch_posts)

    # Exit loop if no more posts are available
    if not batch_posts:
        print("No more posts to fetch.")
        break

    # Optional delay to avoid rate limits
    time.sleep(5)  # Adjust the delay as necessary

# Process the data (example: print the total number of posts fetched)
print(f"Fetched {len(all_posts)} posts in total.")

No more posts to fetch.
Fetched 805 posts in total.


Now that we have collected a large portion of posts/submssions, we'll parse the results and construct a dataframe with this data. We're going to collect more fields from this data than we might need right now, avoiding data limitations in the future.

In [5]:
# Parse are submission objects that we collected.
fields = ('title', 
          'created_utc', 
          'id', 
          'is_original_content', 
          'link_flair_text', 
          'locked',
          'name',
          'num_comments',
          'over_18',
          'permalink',
          'selftext',
          'spoiler',
          'upvote_ratio')
list_of_submissions = []

# Parse each submission into a dictionary of the lised fields.
for submission in all_posts:
    full = vars(submission)
    sub_dict = {field:full[field] for field in fields}
    list_of_submissions.append(sub_dict)

# Create a python dataframe of these submissions.
collected_data = pd.DataFrame.from_records(list_of_submissions)

# Display the dataframe.
display(collected_data)

,title,created_utc,id,is_original_content,link_flair_text,locked,name,num_comments,over_18,permalink,selftext,spoiler,upvote_ratio
0,Everyone that lost money today,1.744257e+09,1jvp0d7,False,Meme,False,t3_1jvp0d7,1,False,/r/wallstreetbets/comments/1jvp0d7/everyone_th...,,False,1.00
1,Is Mango going to pull this same hat trick wit...,1.744257e+09,1jvoy2f,False,Discussion,False,t3_1jvoy2f,2,False,/r/wallstreetbets/comments/1jvoy2f/is_mango_go...,"Mango says major pharma tariffs incoming, phar...",False,0.50
2,MAGA 🫡,1.744257e+09,1jvoy03,False,Meme,False,t3_1jvoy03,4,False,/r/wallstreetbets/comments/1jvoy03/maga/,,False,0.67
3,10/10 would get my face eaten by the bear eate...,1.744256e+09,1jvoqk5,False,Loss,False,t3_1jvoqk5,2,False,/r/wallstreetbets/comments/1jvoqk5/1010_would_...,,False,0.79
4,Volume pumping on NVDA before the tariff news,1.744256e+09,1jvonf2,False,News,False,t3_1jvonf2,8,False,/r/wallstreetbets/comments/1jvonf2/volume_pump...,,False,0.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...
800,EU may delay first counter-tariffs against U.S...,1.742469e+09,1jfmfnk,False,News,False,t3_1jfmfnk,146,False,/r/wallstreetbets/comments/1jfmfnk/eu_may_dela...,"BRUSSELS, March 20 (Reuters) - The European Un...",False,0.93
801,US companies race to secure import tariff exem...,1.742469e+09,1jfmf49,False,News,False,t3_1jfmf49,56,False,/r/wallstreetbets/comments/1jfmf49/us_companie...,March 20 (Reuters) - Washington's temporary re...,False,0.96
802,"Tesla to recall more than 46,000 Cybertrucks d...",1.742466e+09,1jflrx5,False,News,False,t3_1jflrx5,378,False,/r/wallstreetbets/comments/1jflrx5/tesla_to_re...,,False,0.96
803,"Daily Discussion Thread for March 20, 2025",1.742465e+09,1jflc6z,False,Daily Discussion,False,t3_1jflc6z,15006,False,/r/wallstreetbets/comments/1jflc6z/daily_discu...,This post contains content not supported on ol...,False,0.92


### Saving Reddit Data

In [6]:
# Save the collected data to parquet format
PARQUET_PATH = './data/wallstreetbets-collection.parquet'

# Create a pyarrow schema for the data types.
schema = pyarrow.schema([
    ('title', pyarrow.string()), 
    ('created_utc', pyarrow.float64()),
    ('id', pyarrow.string()), 
    ('is_original_content', pyarrow.bool_()), 
    ('link_flair_text', pyarrow.string()), 
    ('locked', pyarrow.bool_()),
    ('name', pyarrow.string()),
    ('num_comments', pyarrow.int64()),
    ('over_18', pyarrow.bool_()),
    ('permalink', pyarrow.string()),
    ('selftext', pyarrow.string()),
    ('spoiler', pyarrow.bool_()),
    ('upvote_ratio', pyarrow.float64()),
])

# If the parqet does not exist, create it.
if not os.path.exists(PARQUET_PATH):
    collected_data.to_parquet(PARQUET_PATH, engine='pyarrow', schema=schema)
    
# If the data file already exist, merge new data with the existing one.
else:
    old_parquet = pd.read_parquet(PARQUET_PATH, engine='pyarrow', schema=schema)
    new_parquet = pd.concat([old_parquet, collected_data])
    new_parquet = new_parquet.drop_duplicates(subset=['id','title','created_utc','name','permalink'], keep='last').reset_index(drop=True)
    new_parquet.to_parquet(PARQUET_PATH, engine='pyarrow', schema=schema)

# Use the new collected data to get comment stuff.
PARQUET_PATH = './data/wallstreetbets-collection.parquet'
submission_collection = pd.read_parquet(PARQUET_PATH, engine='pyarrow', schema=schema)
display(submission_collection)

,title,created_utc,id,is_original_content,link_flair_text,locked,name,num_comments,over_18,permalink,selftext,spoiler,upvote_ratio
0,Tariffs bonanza,1.743624e+09,1jpyan4,False,Discussion,False,t3_1jpyan4,16,False,/r/wallstreetbets/comments/1jpyan4/tariffs_bon...,How will the tariffs affect our market? Are we...,False,0.77
1,🤔,1.743624e+09,1jpy97b,False,Discussion,False,t3_1jpy97b,4,False,/r/wallstreetbets/comments/1jpy97b/_/,,False,0.96
2,"Is It Just Me, or Is Robinhood’s Security Stil...",1.743623e+09,1jpxza5,False,Discussion,False,t3_1jpxza5,8,False,/r/wallstreetbets/comments/1jpxza5/is_it_just_...,I feel like it’s pretty obvious (at least to m...,False,0.83
3,200 puts (~10k): If NVDA goes to $90 this week...,1.743393e+09,1jnuj43,False,YOLO,False,t3_1jnuj43,171,False,/r/wallstreetbets/comments/1jnuj43/200_puts_10...,"If it doesn’t, yall will cover my bet, right?\n",False,0.95
4,All in on Hood,1.743015e+09,1jkjfsa,False,YOLO,False,t3_1jkjfsa,88,False,/r/wallstreetbets/comments/1jkjfsa/all_in_on_h...,Perfect entry today in my opinion. Very simple...,False,0.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1371,EU may delay first counter-tariffs against U.S...,1.742469e+09,1jfmfnk,False,News,False,t3_1jfmfnk,146,False,/r/wallstreetbets/comments/1jfmfnk/eu_may_dela...,"BRUSSELS, March 20 (Reuters) - The European Un...",False,0.93
1372,US companies race to secure import tariff exem...,1.742469e+09,1jfmf49,False,News,False,t3_1jfmf49,56,False,/r/wallstreetbets/comments/1jfmf49/us_companie...,March 20 (Reuters) - Washington's temporary re...,False,0.96
1373,"Tesla to recall more than 46,000 Cybertrucks d...",1.742466e+09,1jflrx5,False,News,False,t3_1jflrx5,378,False,/r/wallstreetbets/comments/1jflrx5/tesla_to_re...,,False,0.96
1374,"Daily Discussion Thread for March 20, 2025",1.742465e+09,1jflc6z,False,Daily Discussion,False,t3_1jflc6z,15006,False,/r/wallstreetbets/comments/1jflc6z/daily_discu...,This post contains content not supported on ol...,False,0.92


## Part _: Getting Comment Data

### Creating a database of reddit threads

In [7]:
# Use the same methofology whih we used to collect submissions, but we'll add a parent submission id. and parent comment id.
# Since the comment section can be very deep, we'll limit comments to a breadth of 10.
# This may still be a lot more comments than we need for larger discussions. 
def extract_comments_for(submission_id: str):
    try:
        submission = reddit.submission(id=submission_id)
        submission.comments.replace_more(limit=10)  # Limit to 10 levels of comments
        comments = []
        
        for comment in submission.comments.list():
            if isinstance(comment, praw.models.MoreComments):
                continue
            
            # NOTE: It looks like the top comment may be a user report. We'll ignore if is has certain text.
            SKIPTEXT = '**User Report**'
            if SKIPTEXT in comment.body:
                continue
            
            # Append the comment data to the list
            comments.append({
                'parent_post_id': submission_id,
                'parent_comment_id': comment.parent_id,
                'comment_id': comment.id,
                'author': str(comment.author),
                'created_utc': comment.created_utc,
                'score': comment.score,
                'body': comment.body
            })
        
        return comments
    
    except Exception as e:
        # Get the HTTP error code if available
        if hasattr(e, 'response') and e.response is not None:
            error_code = e.response.status_code
            print(f"HTTP Error {error_code} while fetching comments for submission {submission_id}")
        else:
            error_code = None
        
        # Print the an erroor message and return nothing.
        print(f"Error fetching comments for submission {submission_id}: {e}")
        return []



In [8]:
# Show the results from one submission's comments
submission_id = submission_collection.iloc[0]['id']

# How many actual comments are there for this submission?
submission = reddit.submission(id=submission_id)
print(f"Submission ID: {submission_id}")
print(f"Title: {submission.title}") 
print(f"Number of comments: {submission.num_comments}")

# Get the comments for the submission
results = extract_comments_for(submission_id)

# Create a dataframe of the comments
comments_df = pd.DataFrame(results)

# Display the comments dataframe
display(comments_df)

Submission ID: 1jpyan4
Title: Tariffs bonanza
Number of comments: 42


,parent_post_id,parent_comment_id,comment_id,author,created_utc,score,body
0,1jpyan4,t3_1jpyan4,ml2y8jb,718cs,1.743624e+09,78,"Everyone is expecting a crash, so expect a big..."
1,1jpyan4,t3_1jpyan4,ml2ygsi,skippy_smooth,1.743624e+09,16,hold onto your butts
2,1jpyan4,t3_1jpyan4,ml2zaxp,pmkhougaz,1.743625e+09,20,Stocks might go down. But they could also go up.
3,1jpyan4,t3_1jpyan4,ml2zuyx,NivvyMiz,1.743625e+09,10,"It says that 8pm, the Markets will ascend to t..."
4,1jpyan4,t3_1jpyan4,ml312zc,dividendvagabond,1.743625e+09,7,"Nah, market is going to 🔥… we’re looking at lo..."
5,1jpyan4,t3_1jpyan4,ml2zgiu,SportsDoc7,1.743625e+09,7,Took some calls today up 120%. Opened some pos...
6,1jpyan4,t3_1jpyan4,ml301ib,nubtraveler,1.743625e+09,5,"They will announce something so outrageous, no..."
7,1jpyan4,t3_1jpyan4,ml32a9d,sirshoelaceman,1.743625e+09,4,Can't wait to break my back working doubles at...
8,1jpyan4,t3_1jpyan4,ml31ft3,eudaimonia_dc,1.743625e+09,3,Has anything of substance actually been announ...
9,1jpyan4,t3_1jpyan4,ml2z2nj,14mmwrench,1.743625e+09,4,Gonna be green as fuk


In [9]:
# Create a pyarrow schema for the data types.
schema = pyarrow.schema([
    ('parent_post_id', pyarrow.string()),
    ('parent_comment_id', pyarrow.string()),
    ('comment_id', pyarrow.string()),
    ('author', pyarrow.string()),
    ('created_utc', pyarrow.float64()),
    ('score', pyarrow.int64()),
    ('body', pyarrow.string())
])

# Collect the comments for all the submissions.
all_comments = []
for submission in submission_collection['id']:
    comments = extract_comments_for(submission)
    all_comments.extend(comments)
    time.sleep(1)  # Optional delay to avoid rate limits
    
# Create a python dataframe of these comments.
comments_df = pd.DataFrame.from_records(all_comments)
display(comments_df)

,parent_post_id,parent_comment_id,comment_id,author,created_utc,score,body
0,1jpyan4,t3_1jpyan4,ml2y8jb,718cs,1.743624e+09,79,"Everyone is expecting a crash, so expect a big..."
1,1jpyan4,t3_1jpyan4,ml2ygsi,skippy_smooth,1.743624e+09,16,hold onto your butts
2,1jpyan4,t3_1jpyan4,ml2zaxp,pmkhougaz,1.743625e+09,22,Stocks might go down. But they could also go up.
3,1jpyan4,t3_1jpyan4,ml2zuyx,NivvyMiz,1.743625e+09,9,"It says that 8pm, the Markets will ascend to t..."
4,1jpyan4,t3_1jpyan4,ml312zc,dividendvagabond,1.743625e+09,8,"Nah, market is going to 🔥… we’re looking at lo..."
...,...,...,...,...,...,...,...
255098,1jfl46v,t1_misk4yp,mj404a3,BillyScheers,1.742627e+09,1,"I like to think that it wouldn't get that bad,..."
255099,1jfl46v,t1_mirubhk,miruisa,throwaway_0x90,1.742467e+09,2,That's kinda what I think is happening here.![...
255100,1jfl46v,t1_mirtt2g,mirtysq,throwaway_0x90,1.742467e+09,1,And so now you feel confident about this new l...
255101,1jfl46v,t1_mirschj,mirst5y,throwaway_0x90,1.742466e+09,2,"where?\n\nAll I see is:\n\n> _""I do think that..."


In [10]:
# Save the parquet file.
PARQUET_PATH = './data/wallstreetbets-comment-collection.parquet'

# Write the comments to parquet file. If it exists, append to it.
if not os.path.exists(PARQUET_PATH):
    comments_df.to_parquet(PARQUET_PATH, engine='pyarrow', schema=schema)
else:
    old_parquet = pd.read_parquet(PARQUET_PATH, engine='pyarrow', schema=schema)
    new_parquet = pd.concat([old_parquet, comments_df])
    new_parquet = new_parquet.drop_duplicates(subset=['parent_post_id','parent_comment_id','author','created_utc'], keep='last').reset_index(drop=True)
    new_parquet.to_parquet(PARQUET_PATH, engine='pyarrow', schema=schema)

## Part _: Analysis of Reddit Data

## Part _: Sentiment Analysis

### Setup Tools

In [11]:
import nltk, re
nltk.download(['punkt',
               'punkt_tab',
               'stopwords',
               'vader_lexicon',
               'names',
               'averaged_perceptron_tagger',
               'wordnet'])

from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /home/bengr/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/bengr/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /home/bengr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/bengr/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package names to /home/bengr/nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/bengr/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/bengr/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Create some basic tools.

In [12]:
# Initialize the Sentiment Intensity Analyzer
sia = SentimentIntensityAnalyzer()
stop_words = set(stopwords.words('english'))

# Function to analyze sentiment of a single comment
def analyze_sentiment(comment):
    # Tokenize the comment
    tokens = word_tokenize(comment.lower())
    
    # Remove stop words
    filtered_tokens = [word for word in tokens if word not in stop_words]
    
    # Get sentiment scores
    sentiment_scores = sia.polarity_scores(' '.join(filtered_tokens))
    
    return sentiment_scores

# Function to clean text
def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Remove special characters
    text = re.sub(r'\@\w+|\#', '', text)
    
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    return text

### Analyse a single submission

In [13]:
# Analyze the sentiment of a single submission.
# We'll look at a Discussion post with the id: '1jpxza5s', talking about robinhoods security
submission = reddit.submission(id='1jpxza5')

# Show the results of the analysis.
sentiment_scores = analyze_sentiment(submission.title)
print(f"Submission: {submission.title}")
print(f"Sentiment Scores: {sentiment_scores}")
print("-" * 80)

Submission: Is It Just Me, or Is Robinhood’s Security Still a Concern?
Sentiment Scores: {'neg': 0.0, 'neu': 0.556, 'pos': 0.444, 'compound': 0.34}
--------------------------------------------------------------------------------
